In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import os
import glob
from torch.utils.data import Dataset, DataLoader, random_split
from timeit import default_timer as timer
from tqdm.auto import tqdm

d:\handgesturedetection\handgesture\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = np.genfromtxt('dataset/keypoint.csv', delimiter=',', skip_header=1)
class0_dir = 'dataset/0'
class1_dir = 'dataset/1'
class2_dir = 'dataset/2'
class3_dir = 'dataset/3'
class4_dir = 'dataset/4'
class5_dir = 'dataset/5'
csv_files_0 = glob.glob(os.path.join(class0_dir, '*.csv'))
csv_files_1 = glob.glob(os.path.join(class1_dir, '*.csv'))
csv_files_2 = glob.glob(os.path.join(class2_dir, '*.csv'))
csv_files_3 = glob.glob(os.path.join(class3_dir, '*.csv'))
csv_files_4 = glob.glob(os.path.join(class4_dir, '*.csv'))
csv_files_5 = glob.glob(os.path.join(class5_dir, '*.csv'))
for csv_file in csv_files_0:
    class0 = pd.read_csv(csv_file)

for csv_file in csv_files_1:
    class1 = pd.read_csv(csv_file)

for csv_file in csv_files_2:
    class2 = pd.read_csv(csv_file)

for csv_file in csv_files_3:
    class3 = pd.read_csv(csv_file)

for csv_file in csv_files_4:
    class4 = pd.read_csv(csv_file)

for csv_file in csv_files_5:
    class5 = pd.read_csv(csv_file)

In [3]:
print(class1)

     0.1  0.2  0.16192560175054704  -0.052516411  0.2910284463894967  \
0      0    0             0.120192     -0.225962            0.062500   
1      0    0             0.113360     -0.246964            0.052632   
2      0    0             0.106164     -0.246575            0.041096   
3      0    0             0.106452     -0.251613            0.045161   
4      0    0             0.106195     -0.238938            0.061947   
..   ...  ...                  ...           ...                 ...   
126    0    0            -0.023179     -0.221854           -0.122517   
127    0    0            -0.021127     -0.232394           -0.126761   
128    0    0            -0.034056     -0.222910           -0.157895   
129    0    0            -0.052147     -0.236196           -0.168712   
130    0    0            -0.031250     -0.190625           -0.146875   

     -0.192560175  0.3982494529540481  -0.319474836  0.5032822757111597  \
0       -0.524038           -0.024038     -0.778846         

In [4]:
#write labels according to the length of each class
labels = np.zeros((len(class0)+len(class1)+len(class2)+len(class3)+len(class4)+len(class5),1))
labels[0:len(class0)] = 0
labels[len(class0):len(class0) + len(class1)] = 1
labels[len(class0) + len(class1):len(class0) + len(class1) + len(class2)] = 2
labels[len(class0) + len(class1) + len(class2):len(class0) + len(class1) + len(class2) + len(class3)] = 3
labels[len(class0) + len(class1) + len(class2) + len(class3):len(class0) + len(class1) + len(class2) + len(class3) + len(class4)] = 4
labels[len(class0) + len(class1) + len(class2) + len(class3) + len(class4):len(class0) + len(class1) + len(class2) + len(class3) + len(class4) + len(class5)] = 5

In [5]:
from typing import Tuple, Union
class CustomDataset(Dataset):
    def __init__(self, data_array, labels):
        self.data_array = data_array
        self.labels = labels

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index) -> Tuple[Union[np.ndarray, torch.Tensor], Union[np.ndarray, torch.Tensor]]:
        data = self.data_array[index]
        label = self.labels[index]
        data = torch.tensor(data)
        label = torch.tensor(label)
        return data, label

In [6]:
BATCH_SIZE = 32
WORKERS = 0

data_array = np.vstack((class0,class1))
data_array = np.vstack((data_array,class2))
data_array = np.vstack((data_array,class3))
data_array = np.vstack((data_array,class4))
data_array = np.vstack((data_array,class5))
dataset = CustomDataset(data_array=data_array,labels=labels)

# Spliting dataset
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=WORKERS)

In [7]:
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")

Training set size: 588
Validation set size: 147


In [8]:
import torch.nn as nn
import torch.nn.functional as F

class ANN(nn.Module):
    def __init__(self, input_size):
        super(ANN, self).__init__()

        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64,6)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p = 0.3)

    def forward(self,x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        
        return x


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size = len(train_dataset[0][0])
model = ANN(input_size=input_size)
model = model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(),
                            lr=0.01)


In [10]:
is_cuda = next(model.parameters()).is_cuda
print(f"Is the model on CUDA? {'Yes' if is_cuda else 'No'}")

Is the model on CUDA? Yes


In [11]:
def accuracy_fn(y_pred,y_true):
    # Get the predicted class by finding the index with the maximum value
    y_pred_class = torch.argmax(y_pred, dim=1)
    
    # Check how many predictions are correct
    correct_predictions = torch.eq(y_pred_class, y_true).sum().item()
    
    # Calculate the accuracy
    accuracy = (correct_predictions / len(y_true)) * 100
    
    return accuracy

In [12]:
epochs = 100
time_start = timer()
for epoch in tqdm(range(epochs)):
    print(f"Epoch = {epoch+1}\n-------------")
    train_loss = 0
    train_acc = 0
    model.train()
    for batch, (X, y) in enumerate(train_loader):
        #frontpropagation
        X = X.type(torch.FloatTensor)   .to(device)       
        y = y.type(torch.LongTensor).squeeze().to(device)
        y_pred = model(X).squeeze()

        #calculate loss and costfunction
        acc = accuracy_fn(y_pred=y_pred,y_true=y)
        loss = loss_fn(y_pred,y)
        train_loss += loss.item()
        train_acc += acc

        #grad to zero
        optimizer.zero_grad()

        #backpropagation
        loss.backward()

        #gradient descent
        optimizer.step()
        if(batch%20 == 0):
            print(f"loss: {loss.item():.5f} ,ACC: {acc:.2f}\n")
    train_loss = train_loss/len(train_loader)
    train_acc = train_acc/len(train_loader)

    #val dataset
    val_loss = 0
    val_acc = 0
    for batch, (X_val_batch, y_val) in enumerate(val_loader):
        X_val_batch = X_val_batch.type(torch.FloatTensor).to(device)
        y_val = y_val.type(torch.LongTensor).squeeze().to(device)
        model.eval()
        with torch.inference_mode():
            y_val_pred = model(X_val_batch).squeeze()
            loss  = loss_fn(y_val_pred,y_val)
            acc = accuracy_fn(y_val_pred,y_val)
            val_loss += loss.item()
            val_acc += acc
    val_loss /=len(val_loader)
    val_acc /=len(val_loader)
    
    print(f"Train loss: {train_loss:.5f} ,Train acc: {train_acc:.2f}\n val loss: {val_loss:.5f}, val acc: {val_acc:.2f}\n")
time_end = timer()
duration = time_end-time_start
print(f"time use:{(duration):2f}seconds")

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch = 1
-------------


  5%|▌         | 5/100 [00:01<00:17,  5.39it/s]

loss: 1.82793 ,ACC: 12.50

Train loss: 1.81258 ,Train acc: 13.38
 val loss: 1.79655, val acc: 16.48

Epoch = 2
-------------
loss: 1.79956 ,ACC: 12.50

Train loss: 1.80594 ,Train acc: 14.04
 val loss: 1.78502, val acc: 16.48

Epoch = 3
-------------
loss: 1.78269 ,ACC: 12.50

Train loss: 1.79159 ,Train acc: 17.43
 val loss: 1.77414, val acc: 17.73

Epoch = 4
-------------
loss: 1.77807 ,ACC: 9.38

Train loss: 1.77980 ,Train acc: 18.64
 val loss: 1.76272, val acc: 27.34

Epoch = 5
-------------
loss: 1.76486 ,ACC: 31.25

Train loss: 1.76821 ,Train acc: 21.49
 val loss: 1.75102, val acc: 38.42

Epoch = 6
-------------
loss: 1.76389 ,ACC: 31.25



 12%|█▏        | 12/100 [00:01<00:06, 13.44it/s]

...

Train loss: 0.30066 ,Train acc: 95.39
 val loss: 0.20358, val acc: 100.00

Epoch = 84
-------------
loss: 0.29882 ,ACC: 96.88

Train loss: 0.32141 ,Train acc: 92.82
 val loss: 0.19496, val acc: 100.00

Epoch = 85
-------------
loss: 0.44007 ,ACC: 84.38

Train loss: 0.31291 ,Train acc: 95.39
 val loss: 0.18714, val acc: 100.00

Epoch = 86
-------------
loss: 0.35603 ,ACC: 93.75

Train loss: 0.30722 ,Train acc: 92.05
 val loss: 0.17924, val acc: 100.00

Epoch = 87
-------------
loss: 0.31569 ,ACC: 96.88

Train loss: 0.28699 ,Train acc: 94.57
 val loss: 0.17220, val acc: 100.00

Epoch = 88
-------------
loss: 0.40212 ,ACC: 87.50

Train loss: 0.26506 ,Train acc: 94.90
 val loss: 0.16572, val acc: 100.00

Epoch = 89
-------------
loss: 0.22650 ,ACC: 96.88



 93%|█████████▎| 93/100 [00:04<00:00, 28.88it/s]

Train loss: 0.27100 ,Train acc: 94.41
 val loss: 0.16045, val acc: 100.00

Epoch = 90
-------------
loss: 0.21690 ,ACC: 96.88

Train loss: 0.25781 ,Train acc: 94.41
 val loss: 0.15368, val acc: 100.00

Epoch = 91
-------------
loss: 0.19805 ,ACC: 96.88

Train loss: 0.26103 ,Train acc: 94.90
 val loss: 0.14901, val acc: 100.00

Epoch = 92
-------------
loss: 0.29559 ,ACC: 96.88

Train loss: 0.22560 ,Train acc: 96.88
 val loss: 0.14325, val acc: 100.00

Epoch = 93
-------------
loss: 0.30594 ,ACC: 87.50

Train loss: 0.22161 ,Train acc: 95.89
 val loss: 0.13737, val acc: 100.00

Epoch = 94
-------------
loss: 0.15061 ,ACC: 100.00

Train loss: 0.23609 ,Train acc: 95.78
 val loss: 0.13248, val acc: 100.00

Epoch = 95
-------------
loss: 0.30345 ,ACC: 93.75

Train loss: 0.23826 ,Train acc: 95.39
 val loss: 0.12872, val acc: 100.00

Epoch = 96
-------------
loss: 0.27650 ,ACC: 93.75



100%|██████████| 100/100 [00:04<00:00, 22.78it/s]

Train loss: 0.22766 ,Train acc: 95.56
 val loss: 0.12359, val acc: 100.00

Epoch = 97
-------------
loss: 0.24623 ,ACC: 100.00

Train loss: 0.23468 ,Train acc: 95.56
 val loss: 0.11936, val acc: 100.00

Epoch = 98
-------------
loss: 0.17108 ,ACC: 96.88

Train loss: 0.20401 ,Train acc: 96.55
 val loss: 0.11388, val acc: 100.00

Epoch = 99
-------------
loss: 0.22598 ,ACC: 96.88

Train loss: 0.22089 ,Train acc: 96.44
 val loss: 0.10958, val acc: 100.00

Epoch = 100
-------------
loss: 0.15984 ,ACC: 96.88

Train loss: 0.18210 ,Train acc: 97.26
 val loss: 0.10668, val acc: 100.00

time use:4.393871seconds
